# Projeto 1 - Ciência dos Dados

Nome: Guilherme Rameh

Nome: Tales Veneruci

Nome: Gabriella Zullo

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [2]:
import os

filename = 'havaiana.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo havaiana.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
train = pd.read_excel(filename)
train.head(5)

Treinamento  Categoria
0  @garotodoflop que dor migo, só apanhei de hava...          0
1  eu amava que no premen eu botava fardinha com ...          0
2  namorada: vc vai assim?\n\neu tentando achar o...          1
3  meu novo perfume é cheiro de havaiana recém fa...          0
4  hj meu dia foi mara \nacordei, fiz um cafézão ...          0

In [4]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

Teste  Categoria
0  @sampaio74 havaiana? não vou nem sujar kkkkkk ...          0
1  eu num sei se é meu pé que ainda tá crescendo ...          0
2                 @mateuscarrilho a havaiana verde 💚          2
3  rt @spn: misha collins de havaiana e com a ban...          2
4  minha havaiana tava preta ele deixou branquinh...          1

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Nosso produto são os chinelos havaianas.<br>
<i>Legenda de classificação:</i>
- Tweet não falava diretamente do produto: <b>Irrelevante</b> 0
- Tweet falava do produto "havaianas" mas não deixava claro o que o consumidor achava deste: <b>Relevante</b> 1
- Tweet falava diretamente que gostou/não gostou de um produto, ou do preço: <b>Muito Relevante</b> 2

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [5]:
train.Categoria = train.Categoria.astype('category')
test.Categoria = test.Categoria.astype('category')

In [6]:
train.Categoria.cat.categories = ['Irrelevante', 'Relevante', 'Muito Relevante']
test.Categoria.cat.categories = ['Irrelevante', 'Relevante', 'Muito Relevante']
train.head()

Treinamento    Categoria
0  @garotodoflop que dor migo, só apanhei de hava...  Irrelevante
1  eu amava que no premen eu botava fardinha com ...  Irrelevante
2  namorada: vc vai assim?\n\neu tentando achar o...    Relevante
3  meu novo perfume é cheiro de havaiana recém fa...  Irrelevante
4  hj meu dia foi mara \nacordei, fiz um cafézão ...  Irrelevante

In [106]:
import re 

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;@/]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

In [107]:
irrelevante = train.loc[train.Categoria=='Irrelevante',:]
relevante = train.loc[train.Categoria=='Relevante',:]
muitoRelevante = train.loc[train.Categoria=='Muito Relevante',:]

In [108]:
trainString_Irrelevante = irrelevante.Treinamento.to_string()
trainString_Relevante = relevante.Treinamento.to_string()
trainString_MuitoRelevante = muitoRelevante.Treinamento.to_string()

tainString_Irrelevante = cleanup(trainString_Irrelevante)
tainString_Relevante = cleanup(trainString_Relevante)
tainString_MuitoRelevante = cleanup(trainString_MuitoRelevante)

In [109]:
todasPalavras_Irrelevante = trainString_Irrelevante.split()
todasPalavras_Relevante = trainString_Relevante.split()
todasPalavras_MuitoRelevante = trainString_MuitoRelevante.split()

serieIrrelevante = pd.Series(todasPalavras_Irrelevante)
serieRelevante = pd.Series(todasPalavras_Relevante)
serieMuitoRelevante = pd.Series(todasPalavras_MuitoRelevante)

In [110]:
tabelaIrrelevante = serieIrrelevante.value_counts()
tabelaRelevante = serieRelevante.value_counts()
tabelaMuitoRelevante = serieMuitoRelevante.value_counts()

In [111]:
tabelaIrrelevanteRelativa = serieIrrelevante.value_counts(True)
tabelaRelevanteRelativa = serieRelevante.value_counts(True)
tabelaMuitoRelevanteRelativa = serieMuitoRelevante.value_counts(True)
print(tabelaRelevanteRelativa.tail(1))
print(tabelaMuitoRelevanteRelativa.tail(1))

@lachefninjaa    0.000718
dtype: float64
283    0.00119
dtype: float64


In [130]:
total_palavras_Irrelevante = len(tabelaIrrelevante)
total_palavras_Relevante = len(tabelaRelevante)
total_palavras_MuitoRelevante = len(tabelaMuitoRelevante)
todas_palavras = total_palavras_Irrelevante + total_palavras_Relevante + total_palavras_MuitoRelevante

In [164]:
def classificador (tweet):
    clean = cleanup(tweet)
    palavras = clean.split()
    probIrrelevante = 1
    probRelevante = 1
    probMuitoRelevante = 1
    for i in palavras:
        if (i in tabelaIrrelevanteRelativa):
            probIrrelevante *= tabelaIrrelevanteRelativa[i]+1/(total_palavras_Irrelevante+todas_palavras)
        else:
            probIrrelevante *= 1/(total_palavras_Irrelevante+todas_palavras)

        if (i in tabelaRelevanteRelativa):
            probRelevante *= tabelaRelevanteRelativa[i]+1/(total_palavras_Relevante+todas_palavras)
        else:
            probRelevante *= 1/(total_palavras_Relevante+todas_palavras)
            
        if (i in tabelaMuitoRelevanteRelativa):
            probMuitoRelevante *= tabelaMuitoRelevanteRelativa[i]+1/(total_palavras_MuitoRelevante+todas_palavras)
        else:
            probMuitoRelevante *= 1/(total_palavras_MuitoRelevante+todas_palavras) 
      
    if probMuitoRelevante > probRelevante and probIrrelevante < probMuitoRelevante:
        test.loc[test.Teste==tweet, 'ClassByTweet'] = 'Muito Relevante'
    elif probIrrelevante < probRelevante and probRelevante > probMuitoRelevante:
        test.loc[test.Teste==tweet, 'ClassByTweet'] = 'Relevante'
    else:
        test.loc[test.Teste==tweet, 'ClassByTweet'] = 'Irrelevante'

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [165]:
for e in test.Teste:
    classificador(e)

In [166]:
test.ClassByTweet = test.ClassByTweet.astype('category')
test.ClassByTweet.value_counts()

Muito Relevante    153
Relevante          121
Irrelevante         53
Name: ClassByTweet, dtype: int64

In [167]:
test['ClassByTweet'].cat.categories = ['Irrelevante', 'Relevante', 'Muito Relevante']

In [168]:
pd.crosstab(test.Categoria, test.ClassByTweet, normalize=True)

ClassByTweet     Irrelevante  Relevante  Muito Relevante
Categoria                                               
Irrelevante         0.137615   0.232416         0.192661
Relevante           0.021407   0.073394         0.149847
Muito Relevante     0.003058   0.064220         0.125382

In [169]:
pd.crosstab(test.Categoria, test.ClassByTweet, normalize='columns')

ClassByTweet     Irrelevante  Relevante  Muito Relevante
Categoria                                               
Irrelevante         0.849057   0.628099         0.411765
Relevante           0.132075   0.198347         0.320261
Muito Relevante     0.018868   0.173554         0.267974

___
### Concluindo

In [64]:
print(f'Classificados corretamente Irrelevantes: 13.7%')
print(f'Classificados corretamente Relevantes: 7.3%')
print(f'Classificados corretamente Muito Relevantes: 12.5%')
print(f'\nClassificados corretamente geral: {13.7+7.3+12.5:.2f}%')

Classificados corretamente Irrelevantes: 13.7%
Classificados corretamente Relevantes: 7.3%
Classificados corretamente Muito Relevantes: 12.5%

Classificados corretamente geral: 33.50%


Dado que muitos tweets são difíceis de classificar, uma vez que determinar a probabilidade pelas palavras é diferente do que a interpretação geral pode transmitir, é muito mais fácil conseguir um sucesso maior em classificar os tweets Irrelevantes do que as outras categorias. Além disso, uma pequena modificação realizada no classificador provou-se melhorar a performance do mesmo: me refiro ao else dentro do for. O que esta lógica provoca é que, caso a palavra não esteja em nenhuma das listas (ou seja, não foi usada em nenhum tweet do treinamento), é muito mais provável que ela seja Irrelevante. Porém, em vez de multiplicar a probabilidade por 0, o que automaticamente classificaria esse tweet como Irrelevante (as probabilidades "Muito Relevante" e "Relevante" seriam anuladas), foi usado o método de *laplace* que impede que essas probabilidades sejam anuladas. Isso permite, caso haja uma outra palavra que seja muito mais provavel de classificar ou em Relevante ou em Muito Relevante, o classificador ainda tem chance de classificar o tweet "corretamente".

O classificador Naïve Bayes pode ser usado em diversos outros cenários, por exemplo classificar um perfil de estudante mais provavel de ficar de DP ou classificar o tipo de consumidor mais provavel de pedir fastfood por aplicativos de entrega. Claro,  isso são apenas exemplos, mas de maneira geral, o classificador Naïve Bayes tem maior eficiência quando se trata da identificação de dados numéricos, e não textos, já que a multiplicação das probabilidades das palavras pode resultar um uma probabilidade que não representa o significado do texto, também pois esse pode variar dependendo das diversas interpretações (algo que esse classificador não é capaz de fazer).

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**